# Cross Validation Training

Create Cross-Validation Splits

In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from collections import Counter
import argparse

# Setting up command-line argument parsing
# parser = argparse.ArgumentParser(description='Perform stratified K-Fold on object detection dataset.')
# parser.add_argument('csv_file', type=str, help='Path to the CSV dataset file')
# args = parser.parse_args()

csv_file = "annotations_2488_mlflow_shuffled_n.csv"
# Load the dataset from the provided file path
df = pd.read_csv(csv_file)

# Correcting the column names based on your dataset format
df.columns = ['Split', 'ImagePath', 'Label', 'Other', 'Columns', 'Not', 'Needed', 'For', 'This', 'Calculation', 'Wow']

# Group by ImagePath to ensure all labels for an image stay together
grouped = df.groupby('ImagePath')

# Use the most frequent label in each image for stratification
image_class_counts = grouped['Label'].apply(lambda x: Counter(x).most_common(1)[0][0]).reset_index(name='MostCommonLabel')

# Stratified K-Fold Cross Validation
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
folds = {}
fold_files = []
num_imgs = 0
for fold, (train_idx, test_idx) in enumerate(skf.split(image_class_counts['ImagePath'], image_class_counts['MostCommonLabel'])):
    test_image_paths = image_class_counts.iloc[test_idx]['ImagePath'].tolist()
    fold_data = df[df['ImagePath'].isin(test_image_paths)]
    folds[fold] = fold_data
    num_imgs = num_imgs + fold_data['ImagePath'].nunique()
    # Print the distribution of each fold and the number of unique images
    print(f"Fold {fold}:")
    print(f"Number of unique images: {fold_data['ImagePath'].nunique()}")
    print(f"Number of images: {fold_data['ImagePath'].count()}")
    print("Label Distribution:")
    print(fold_data['Label'].value_counts())
    print()

# Creating CSV files and calculating class distributions
class_distributions = {}

for fold, validation_data in folds.items():
    # Combine the other folds to form the training data
    train_data = pd.concat([folds[f] for f in folds if f != fold])

    # Marking the validation and training data
    validation_data['Split'] = 'VALIDATE'
    train_data['Split'] = 'TRAIN'

    # Combine training and validation data
    combined_data = pd.concat([train_data, validation_data])

    # Save to CSV
    filename = f'{num_imgs}_cv_fold_{fold}.csv'
    fold_files.append(filename)
    combined_data.to_csv(filename, index=False, header=False)

    # Collect class distributions for the validation fold
    class_distributions[f'Fold {fold}'] = validation_data['Label'].value_counts()

# Convert class distributions to a DataFrame and save
class_distribution_df = pd.DataFrame(class_distributions)
class_distribution_df.to_csv(f'{num_imgs}_class_distributions.csv')


Fold 0:
Number of unique images: 937
Number of images: 4146
Label Distribution:
other          823
Tomato         389
Bell-Pepper    320
Onion          302
Garlic         237
Potato         209
Lemon          180
Carrot         180
Cucumber       169
Egg            157
Chilli         147
Zucchini       142
Apple          125
Scallion       117
Ginger         109
Lime           105
Banana          99
Avocado         92
Pumpkin         68
Cabagge         64
Eggplant        40
Broccoli        31
Mango           26
Cauliflower     15
Name: Label, dtype: int64

Fold 1:
Number of unique images: 937
Number of images: 4084
Label Distribution:
other          833
Tomato         386
Onion          299
Bell-Pepper    278
Garlic         215
Egg            193
Lemon          176
Cucumber       174
Potato         170
Carrot         164
Chilli         151
Zucchini       139
Apple          128
Banana         122
Scallion       120
Ginger         101
Lime            98
Avocado         82
Pumpkin        

/tmp/ipykernel_240/2726608897.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_data['Split'] = 'VALIDATE'
/tmp/ipykernel_240/2726608897.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  validation_data['Split'] = 'VALIDATE'
/tmp/ipykernel_240/2726608897.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/u

Train model with cross validation

In [1]:
import numpy as np
import os
import json

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

# The current code uses Focal loss which has already weighted loss because of alpha and gamma
model_name = 'efficientdet-lite0' # EfficientDetLite1Spec must also be set accordingly!
epochs = 30
batch_size = 20

train_only_fold_number = 0 

fold_files = ['4684_cv_fold_0.csv','4684_cv_fold_1.csv','4684_cv_fold_2.csv','4684_cv_fold_3.csv','4684_cv_fold_4.csv']

for fold_i, fold_file in enumerate(fold_files):
    custom_model_dir_name = 'model_'+"2488_more_classes_plus_indiv"#str(num_distinct_files)
    model_dir = f"models/{model_name}/{custom_model_dir_name}_e{str(epochs)}_b{str(batch_size)}_cvf_{fold_i}"
    if train_only_fold_number is not None:
        if fold_i == train_only_fold_number:
            print(f"training only fold number {train_only_fold_number}")
        else:
            print(f"skipping fold number {fold_i}")
            continue
    print(f"training for fold number {fold_i} with file {fold_file}")
    #spec = model_spec.get('efficientdet_lite1')
    # check this url to check valid hparam values
    # https://github.com/tensorflow/examples/blob/master/tensorflow_examples/lite/model_maker/third_party/efficientdet/hparams_config.py
    spec = object_detector.EfficientDetLite0Spec( # change this also to correct model spec
        model_name = model_name,
        model_dir='/home/alex/checkpoints/',
        hparams='grad_checkpoint=true,strategy=gpus',
        epochs=epochs, batch_size=batch_size,
        steps_per_execution=1, moving_average_decay=0,
        var_freeze_expr='(efficientnet|fpn_cells|resample_p6)',
        tflite_max_detections=25
    )
    
    train_data, validation_data, test_data = object_detector.DataLoader.from_csv(fold_file)
    
    model = object_detector.create(train_data, model_spec=spec, train_whole_model=True, validation_data=validation_data)
    
    if not os.path.exists(model_dir):
        os.makedirs(model_dir)

    label_map = model.model_spec.config.label_map.as_dict()
    # Writing the dictionary to a JSON file
    with open(model_dir+'/label_map.json', 'w') as file:
        json.dump(label_map, file)
        
        
    model.evaluate(validation_data)
    
    model.export(export_dir=model_dir)
    print(f"exported to model to {model_dir}")
    
    model.evaluate_tflite(model_dir+'/model.tflite', validation_data)


/home/alex/anaconda3/envs/conda_env/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.4 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


training only fold number 0
training for fold number 0 with file 4684_cv_fold_0.csv


2024-01-07 23:52:24.863032: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-07 23:52:24.891999: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-07 23:52:24.892055: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:922] could not open file to read NUMA node: /sys/bus/pci/devices/0000:09:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-01-07 23:52:24.893493: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

Epoch 1/30


2024-01-07 23:52:51.090177: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101


...

Estimated count of arithmetic ops: 1.834 G  ops, equivalently 0.917 G  MACs


fully_quantize: 0, inference_type: 6, input_inference_type: 3, output_inference_type: 0
2024-01-08 00:46:39.943644: I tensorflow/compiler/mlir/lite/flatbuffer_export.cc:1963] Estimated count of arithmetic ops: 1.834 G  ops, equivalently 0.917 G  MACs



exported to model to models/efficientdet-lite0/model_2488_more_classes_plus_indiv_e30_b20_cvf_0


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


937/937 [==============================] - 1247s 1s/step

skipping fold number 1
skipping fold number 2
skipping fold number 3
skipping fold number 4


In [2]:
model.evaluate(validation_data)

47/47 [==============================] - 27s 421ms/step



{'AP': 0.11482489,
 'AP50': 0.23367226,
 'AP75': 0.10126301,
 'APs': 0.0,
 'APm': 0.07334023,
 'APl': 0.13094635,
 'ARmax1': 0.120915554,
 'ARmax10': 0.23591745,
 'ARmax100': 0.25812423,
 'ARs': 0.0,
 'ARm': 0.12904899,
 'ARl': 0.29629952,
 'AP_/Onion': 0.08595568,
 'AP_/Apple': 0.07393514,
 'AP_/Banana': 0.18724176,
 'AP_/Pumpkin': 0.2047163,
 'AP_/other': 0.27458826,
 'AP_/Tomato': 0.099563606,
 'AP_/Scallion': 0.043484483,
 'AP_/Cucumber': 0.076655135,
 'AP_/Lime': 0.102669716,
 'AP_/Potato': 0.09205252,
 'AP_/Garlic': 0.0973324,
 'AP_/Carrot': 0.07163637,
 'AP_/Bell-Pepper': 0.12691122,
 'AP_/Zucchini': 0.036796402,
 'AP_/Eggplant': 0.1459331,
 'AP_/Chilli': 0.04162203,
 'AP_/Lemon': 0.10796707,
 'AP_/Avocado': 0.040056285,
 'AP_/Mango': 0.07067793,
 'AP_/Egg': 0.29487553,
 'AP_/Ginger': 0.10812945,
 'AP_/Cabagge': 0.21321613,
 'AP_/Broccoli': 0.060286965,
 'AP_/Cauliflower': 0.09949399}